In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
from torch.utils.data import Dataset, random_split
import torchvision.io as io
from torchvision import transforms 
import torch
import torchvision
from torch import nn

# *Set up environment* 

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


/home/karim/anaconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [6]:
if "google.colab" in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True, use_metadata_server=False)

In [7]:
if "google.colab" in sys.modules:
    import requests
    # Save datagenerators as file to colab working directory
    # If you are using GitHub, make sure you get the "Raw" version of the code
    url = 'https://raw.githubusercontent.com/karimassi/road-segmentation/main/src/training.py?token=AFUMR6ANI6H6PQ6BZGQQI4C72E32O'
    r = requests.get(url)

    with open('training.py', 'w') as f:
        f.write(r.text)

    url = 'https://raw.githubusercontent.com/karimassi/road-segmentation/main/src/MyDataSet.py?token=AFUMR6FXF63DZ6R24EZXE4K72E33E'
    r = requests.get(url)
        
    with open('MyDataSet.py', 'w') as f:
        f.write(r.text)
        
    import MyDataSet
    import training 
else:
    from src import training   
    from src import MyDataSet

# *Baseline CNN Model*

In [6]:
threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

class PatchModel(nn.Module):
    """
    Model that tells if a 16 x 16 RGB (as a 3 x 16 x 16 tensor) correspond to a road (1) or not (0)
    """
    def __init__(self):
        super().__init__()
        
        # 3 channels 16 x 16
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=5
        )

        # 32 channels 12 x 12 (12 = 16 - (kernel_size - 1))
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        # 32 channels 6 x 6 (6 = 12 / kernel_size)
        self.conv2 = nn.Conv2d(
            in_channels=32,
            out_channels=64,
            kernel_size=3
        )

        # 64 channels 4 x 4 (4 = 6 - (kernel_size - 1))
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        
        # 64 channels 2 x 2 (2 = 4 / kernel_size)
        self.lin1 = nn.Linear(
            in_features=64*2*2,
            out_features= 512
        )
        self.lin2 = nn.Linear(
            in_features=512,
            out_features=1
        )
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.pool1(self.relu(self.conv1(x)))
        x = self.pool2(self.relu(self.conv2(x)))
        x = x.view(-1, 64*2*2)
        x = self.relu(self.lin1(x))
        x = self.sigmoid(self.lin2(x))
        # If we are in testing mode then the output should be either 0 or 1
        if not self.training:
            x = 1 * (x > threshold)
        return x.view(-1)

# *Load the data*

In [ ]:
batch_size = 100

dataset = MyDataSet.PatchedSatImagesDataset(MyDataSet.img_path, MyDataSet.gt_path, threshold)

rotations = [(45, 45), (90, 90), (135, 135), (180, 180), (-45, -45), (-90, -90), (-135, -135), (-180, -180)]
for rotation in rotations:
    dataset += MyDataSet.PatchedSatImagesDataset(MyDataSet.img_path, MyDataSet.gt_path, threshold, transforms.RandomRotation(rotation))
    
dataset += MyDataSet.PatchedSatImagesDataset(MyDataSet.img_path, MyDataSet.gt_path, threshold, transforms.RandomVerticalFlip(p = 1.0))
dataset += MyDataSet.PatchedSatImagesDataset(MyDataSet.img_path, MyDataSet.gt_path, threshold, transforms.RandomHorizontalFlip(p = 1.0))

data_len = len(dataset)
train_len = round(data_len * 0.7)
test_len = round(data_len * 0.3)

dataset_train, dataset_test = random_split(dataset, [train_len, test_len])

print(len(dataset_train), len(dataset_test))

dataloader_train = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=True
)

# *Train the model*

In [ ]:
num_epochs = 100
learning_rate = 1e-2
decay_rate = 0.95

# Train the logistic regression model with the Adam optimizer
criterion = torch.nn.BCELoss()
model = PatchModel().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decay_rate, verbose=True)
training.train(model, criterion, dataloader_train, dataloader_test, optimizer, scheduler, num_epochs)

# *Predict output for testing images*

In [28]:
submission = MyDataSet.PatchedTestSatImagesDataset(MyDataSet.test_path)
dataloader_submission = torch.utils.data.DataLoader(
    submission, 
    batch_size=1,
    shuffle=False
)
with open('submission.csv', 'w') as f:
    f.write('id,prediction\n')
    model.eval()
    for img_id, X in dataloader_submission:
        X = X.to(device)
        Y = model(X).item()
        f.write(f'{img_id[0]},{Y}\n')        